In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

## Task1

빈 칸을 채워주세요!

단계별 output이 github 파일에는 남아있으니 그 output과 동일한 형태인지 확인하면서 진행해주시면 됩니다~

In [8]:
#1. 생성할 문장 데이터

sentence = ("Brick walls are there for a reason and you must not think "
            "that the brick walls aren't there to keep us out, but rather "
            "in this way that the brick walls are there to show us how badly we want things.")

In [9]:
#2. 문자 집합 만들기
world_set = list(set(sentence))

## 문제(1): 각 문자에 정수 인코딩 (공백도 하나의 원소로 포함)
vocab = {word: i for i, word in enumerate(world_set)}

In [10]:
print(vocab)

{',': 0, 'm': 1, 'k': 2, 'h': 3, 'e': 4, 't': 5, ' ': 6, 'l': 7, 'u': 8, 'y': 9, 'd': 10, 'n': 11, 'o': 12, 'B': 13, 's': 14, 'g': 15, 'f': 16, 'r': 17, 'c': 18, 'i': 19, 'w': 20, 'a': 21, 'b': 22, 'p': 23, '.': 24, "'": 25}


In [11]:
#3. 문자 집합 크기 확인

vocab_size = len(vocab)
print('문자 집합 크기 : {}'.format(vocab_size))

문자 집합 크기 : 26


In [12]:
#4. 하이퍼 파라미터 설정(자유롭게 수정해보세요!)

hidden_size = vocab_size # 같아야 하는 것 확인!
sequence_length = 12  # 너무 길거나 너무 짧게 잡으면 안됩니다!
learning_rate = 0.01

In [13]:
#5. seqence 길이 단위 자르기

# 데이터 구성을 위한 리스트
x_data = []
y_data = []

## 문제(2): 반복문 내에서의 인덱싱을 사용하여 sequence_length 값 단위로 샘플을 잘라 데이터 만들기, y_str은 x_str은 한 칸씩 쉬프트된 sequnce

for i in range(0, len(sentence) - sequence_length):
  x_str = sentence[i: i + sequence_length]
  y_str = sentence[i+1: i + sequence_length+1]
  print(i, x_str, "->", y_str)

  # x_str과 y_str이 문자집합에 해당하는 인덱스를 각각 x_data, y_data에 append
  x_data.append([vocab[c] for c in x_str])
  y_data.append([vocab[d] for d in y_str])

0 Brick walls  -> rick walls a
1 rick walls a -> ick walls ar
2 ick walls ar -> ck walls are
3 ck walls are -> k walls are 
4 k walls are  ->  walls are t
5  walls are t -> walls are th
6 walls are th -> alls are the
7 alls are the -> lls are ther
8 lls are ther -> ls are there
9 ls are there -> s are there 
10 s are there  ->  are there f
11  are there f -> are there fo
12 are there fo -> re there for
13 re there for -> e there for 
14 e there for  ->  there for a
15  there for a -> there for a 
16 there for a  -> here for a r
17 here for a r -> ere for a re
18 ere for a re -> re for a rea
19 re for a rea -> e for a reas
20 e for a reas ->  for a reaso
21  for a reaso -> for a reason
22 for a reason -> or a reason 
23 or a reason  -> r a reason a
24 r a reason a ->  a reason an
25  a reason an -> a reason and
26 a reason and ->  reason and 
27  reason and  -> reason and y
28 reason and y -> eason and yo
29 eason and yo -> ason and you
30 ason and you -> son and you 
31 son and you  ->

In [14]:
# 출력해서 한 칸씩 쉬프트된 것 확인하기!

print(x_data[0])
print(y_data[0])

[13, 17, 19, 18, 2, 6, 20, 21, 7, 7, 14, 6]
[17, 19, 18, 2, 6, 20, 21, 7, 7, 14, 6, 21]


In [16]:
##6. 입력 시퀀스에 대해 원핫인코딩 수행

## 문제(4) : x_data를 원핫인코딩 > numpy의 eye를 쓸 수 있지 않을까?
x_one_hot = np.eye(vocab_size)[x_data]

##7. 입력 데이터, 레이블데이터 텐서로 변환

## 문제(5) : x_one_hot과 y_data 텐서로 변환 : 둘 다 같은 형식의 텐서로 변환하면 될까?? (FloatTensor, LongTesor 중 맞는 것은?)
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

In [17]:
##8. 크기 확인
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([186, 12, 26])
레이블의 크기 : torch.Size([186, 12])


In [18]:
##9.원핫인코딩 결과 샘플 확인하기
print(X[0])

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0.,

In [19]:
##10. 레이블 데이터 샘플 확인하기
print(Y[0])

tensor([17, 19, 18,  2,  6, 20, 21,  7,  7, 14,  6, 21])


In [25]:
##11. RNN 모델 구현

##문제(6) : 기본 pytorch 인자 넣기 연습 + forward 채우기
### 조건 : rnn layer 2개 쌓기 + 마지막은 fc layer
### batch_fisrt 설정 필요할까? (유튜브 강의 참고)

class Net(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, layers):
        super(Net, self).__init__()
        self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers = layers, batch_first = True)
        self.fc = torch.nn.Linear(hidden_dim, hidden_dim, bias = True)
  
    def forward(self, x):
        x, state = self.rnn(x)
        x = self.fc(x)
        return x

In [26]:
net = Net(vocab_size, hidden_size, 2)

In [27]:
##12. loss function
criterion = torch.nn.CrossEntropyLoss()

##13. optimizer
optimizer = optim.Adam(net.parameters(), learning_rate)

In [28]:
##14. 출력 크기 점검
outputs = net(X)
print(outputs.shape)

torch.Size([186, 12, 26])


In [30]:
##15. Training 시작

for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    ##문제(7) : outputs, Y 형태 그대로 넣으면 안되죠. view 함수를 이용해 loss값을 계산해봅시다.
    loss = criterion(outputs.view(-1, vocab_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([world_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += world_set[result[-1]]

    print(predict_str)

..rfffrfffrfpfiiprpfiipffrprfi.pfrrppfrfpffpfpuipfuifufrfipppfruuprffffffrfffrfpfi.frrpupfiipprfiifrfrfprfirpfuiffu.pfrfrrrrfrfrffrf.pfruuprffffffrfffrfpfiiprpfiipprpfprffrfpprfppfffrrbrrfrrrpufrfr
.rrrrrrrrrrrrrrrrrrrrrrrrrsrsrsrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrsrsrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrr
rrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrr
rrr rrrrr  rrr rrr r rr rr rr  r rrrr rrrrrr   rrrrrrrr rrrrrrrrr r rr rrrrr  rrr  rrrrr r rrrrr   rrrrrr rr r rr  r rrr rrrr rrrrrrrrrrrr r rr rrrrr  rrr rrr r rrrr  rrrr r rrrr  rrrr rrrrrrrrr rr
r                                                                                                                                                                                                    
          

In [31]:
predict_str

"rick walls are there tor a reason and you must not think that the brick walls are 't there to seep us out, but rather in this way that the brick walls are there to show us how badly we want thinks'"

결과가 어떤가요?? 마지막 에폭의 문장이 그럴싸한가요?

## Task2

위 sentence는 제가 임의로 생성한 문장들입니다.

마음에 드시는 문구 가져오셔서 문장이 어떻게 생성되는지 확인해보세요! 

영어가 아닌 한국어로 시도해보는 것도 좋겠죠? 

수정이 많이 필요(토큰화 등) 할 수 있으나 한번 시도해보시는 것 권장드립니다 :)

위 베이스라인은 어디든 수정하셔도 좋고 조금 더 자연스러운 문장이 나올 수 있게 다양한 시도를 해보세요!

조건 : 문장 3개 이상, 연결성이 있는 문장을 " " 으로 구분하여 ( )에 넣기

In [160]:
#1. 생성할 문장 데이터

sentence = ("Child undernutrition is a major public health problem."
"One third of all undernourished children globally reside in Sub-Saharan Africa (SSA)."
"The factors that predispose a child to undernutrition are multisectoral." 
"To yield a sustainable improvement in child nutrition in SSA, a holistic multi-strategy community-based approach is needed.")

In [161]:
#2. 문자 집합 만들기
world_set = list(set(sentence))

## 문제(1): 각 문자에 정수 인코딩 (공백도 하나의 원소로 포함)
vocab = {c: i for i,c in enumerate(world_set)}
print(world_set,vocab)

[',', 'C', 'm', 'h', 'e', 't', ' ', 'l', 'u', 'y', 'T', '-', 'n', 'd', 'A', '(', 'o', 'v', 's', 'g', 'r', 'f', ')', 'c', 'j', 'i', 'O', 'a', 'p', 'b', '.', 'S'] {',': 0, 'C': 1, 'm': 2, 'h': 3, 'e': 4, 't': 5, ' ': 6, 'l': 7, 'u': 8, 'y': 9, 'T': 10, '-': 11, 'n': 12, 'd': 13, 'A': 14, '(': 15, 'o': 16, 'v': 17, 's': 18, 'g': 19, 'r': 20, 'f': 21, ')': 22, 'c': 23, 'j': 24, 'i': 25, 'O': 26, 'a': 27, 'p': 28, 'b': 29, '.': 30, 'S': 31}


In [162]:

#3. 문자 집합 크기 확인

vocab_size = len(vocab)
print('문자 집합 크기 : {}'.format(vocab_size))

문자 집합 크기 : 32


In [163]:

#4. 하이퍼 파라미터 설정(자유롭게 수정해보세요!)

hidden_size = vocab_size # 같아야 하는 것 확인!
sequence_length = 15  # 너무 길거나 너무 짧게 잡으면 안됩니다!
learning_rate = 0.01

In [164]:

#5. seqence 길이 단위 자르기

# 데이터 구성을 위한 리스트
x_data = []
y_data = []

## 문제(2): 반복문 내에서의 인덱싱을 사용하여 sequence_length 값 단위로 샘플을 잘라 데이터 만들기, y_str은 x_str은 한 칸씩 쉬프트된 sequnce
## 아래 결과 코드를 보면 x와 y의 관계가 한칸 차이임을 알 수 있다.
for i in range(0, len(sentence) - sequence_length):
  x_str = sentence[i:i+sequence_length]
  y_str = sentence[i+1:i+sequence_length+1]
  print(i, x_str, "->", y_str)

  # x_str과 y_str이 문자집합에 해당하는 인덱스를 각각 x_data, y_data에 append
  x_data.append([vocab[c] for c in x_str])
  y_data.append([vocab[d] for d in y_str])

0 Child undernutr -> hild undernutri
1 hild undernutri -> ild undernutrit
2 ild undernutrit -> ld undernutriti
3 ld undernutriti -> d undernutritio
4 d undernutritio ->  undernutrition
5  undernutrition -> undernutrition 
6 undernutrition  -> ndernutrition i
7 ndernutrition i -> dernutrition is
8 dernutrition is -> ernutrition is 
9 ernutrition is  -> rnutrition is a
10 rnutrition is a -> nutrition is a 
11 nutrition is a  -> utrition is a m
12 utrition is a m -> trition is a ma
13 trition is a ma -> rition is a maj
14 rition is a maj -> ition is a majo
15 ition is a majo -> tion is a major
16 tion is a major -> ion is a major 
17 ion is a major  -> on is a major p
18 on is a major p -> n is a major pu
19 n is a major pu ->  is a major pub
20  is a major pub -> is a major publ
21 is a major publ -> s a major publi
22 s a major publi ->  a major public
23  a major public -> a major public 
24 a major public  ->  major public h
25  major public h -> major public he
26 major public he -> 

In [121]:

# 출력해서 한 칸씩 쉬프트된 것 확인하기!

print(x_data[0])
print(y_data[0])

[1, 3, 25, 7, 13, 6, 8, 12, 13, 4, 20, 12, 8, 5, 20]
[3, 25, 7, 13, 6, 8, 12, 13, 4, 20, 12, 8, 5, 20, 25]


In [165]:

x_one_hot = [np.eye(len(vocab))[x] for x in x_data]

##7. 입력 데이터, 레이블데이터 텐서로 변환

## 문제(5) : x_one_hot과 y_data 텐서로 변환 : 둘 다 같은 형식의 텐서로 변환하면 될까?? (FloatTensor, LongTesor 중 맞는 것은?)
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

In [166]:
    

##8. 크기 확인
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([319, 15, 32])
레이블의 크기 : torch.Size([319, 15])


In [167]:

##10. 레이블 데이터 샘플 확인하기
print(Y[0])

tensor([ 3, 25,  7, 13,  6,  8, 12, 13,  4, 20, 12,  8,  5, 20, 25])


In [130]:
class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_size=input_dim,hidden_size=hidden_dim,batch_first=True)
    self.fc = torch.nn.Linear(hidden_dim,hidden_dim,bias=True)
  
  def forward(self, x):
    x,_status = self.rnn(x)
    x=self.fc(x)
    return x
     

In [131]:
net = Net(vocab_size, hidden_size, 2)

In [132]:

##12. loss function
criterion = torch.nn.CrossEntropyLoss()

##13. optimizer
optimizer = optim.Adam(net.parameters(), learning_rate)

In [133]:

##15. Training 시작

for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    ##문제(7) : outputs, Y 형태 그대로 넣으면 안되죠. view 함수를 이용해 loss값을 계산해봅시다.
    loss = criterion(outputs.view(-1, vocab_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([world_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += world_set[result[-1]]

    print(predict_str)

aaaa))a)))))b)))))))))))))b))))))))))a)))))a))))))))a))))))))l)))))))C)b))))))b))))))))))))a)))))))))))))))))a)))))))))))a))))a)))))))))a))))))))a)))))))))))))))))))))))a))))))))b)))))b))))))))))))bb)))))a))))))))))))a)))))a)))))a)))))b))))) ))))))))))))))b)))))))))))))a))))))))aa)))bb))))))))))))))) bb)))))))))))))))))a)))))))))))
aaaaaaaaaaaaaae)a))aaaaalaaaaaeaaaaaaaaaaaaaaaaeaaaaaa aaaaaaeaaa aaaaaaaaaaaa eaaaaaaaaaaaaaaaau aaaaaaaaaa)aaaaaaaa aaaaalaaaaeaaaa aaaaaaaaaaaaa)eaaaaaa)aeaaaaa)aaaaaaaaaaaa) aaaaaaaae)a))aaleaaaaua)aaaa)elaaaaaaaaaaaaaaaaa))aaaaaaaaaea aaa a))aaaaaaaaaaae)a))aaaaaaaaa aaaaaaaa)aaaaua) aaelaa aaaa aa aaa aaaaaala ealaaaaaaaaaaaa
aaaaaaa aa  aa raa  aaaalaaraa a   aaaaaa aaaa    aa a  a aaa aaa ala aa an  a  aaaaaaaaa aaa a   a a ananaaeaaa aa a   aaalaaa eaaaa a aaaaaa  laa  aaaaaa   naaa  aaaanaaa aaa  a an  aa raa  alena auaa aaa  lu aa aanaaaanaaaala aaaaaa       n ana aaaa aa aa aaa  aa aa aa aaaa  aaraaaauaa aa lan   aa aa aan a aaaal     aaaaaaaaaaa

In [134]:

predict_str
     

'hild nndernutrition in a major public health problem.One third of all undernuurished children globally reside in Sub-Saharan Africa (SSA).The factors that predispose a child to undernutrition ire multisectoral.To yield a sustainable improvement in child nutrition in SSA, a holistic multi-strategy community-based approach is n  dedi'

In [ ]:
자음 없어지는 경우 ㅇ, 띄어쓰기가 잘 안됨
One hot 말고 다른 인코딩을 사용한다면 더 나은 결과가 나올지? 

In [137]:
import nltk
nltk.download('punkt')
from nltk import sent_tokenize
sents=sent_tokenize(sentence)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\82107\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


In [138]:
sents[0]

'Child undernutrition is a major public health problem.One third of all undernourished children globally reside in Sub-Saharan Africa (SSA).The factors that predispose a child to undernutrition are multisectoral.To yield a sustainable improvement in child nutrition in SSA, a holistic multi-strategy community-based approach is needed.'

In [156]:
#CountVectorizer:
from sklearn.feature_extraction.text import CountVectorizer
# list of text documents
text=['Child undernutrition is a major public health problem.' 
      'One third of all undernourished children globally reside in Sub-Saharan Africa (SSA).'
      'The factors that predispose a child to undernutrition are multisectoral.'
      'To yield a sustainable improvement in child nutrition in SSA, a holistic multi-strategy community-based approach is needed.']
# create the transform
vectorizer = CountVectorizer()
# tokenize and build vocab
vectorizer.fit(x_data)
# summarize
print("vocabulary:{}\n".format(vectorizer.vocabulary_))
# encode document
vector = vectorizer.fit_transform(x_data)
# summarize encoded vector
print("vector size:{}\n".format(vector.shape))
print(type(vector),"\n")
print("vector:\n{}".format(vector.toarray()))

AttributeError: 'list' object has no attribute 'lower'

In [150]:
vector= vector.toarray()

In [171]:
vector

array([[1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 3, 2, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1]], dtype=int64)

In [172]:
## 문제(5) : x_one_hot과 y_data 텐서로 변환 : 둘 다 같은 형식의 텐서로 변환하면 될까?? (FloatTensor, LongTesor 중 맞는 것은?)
X = torch.FloatTensor(vector)
Y = torch.LongTensor(y_data)

In [173]:

##8. 크기 확인
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([1, 38])
레이블의 크기 : torch.Size([319, 15])
